# Filtering

In [ ]:
!pip install xlsxwriter io
!pip install --upgrade pandas

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
from io import BytesIO
import xlsxwriter

In [ ]:
df = pd.read_csv('/content/central-tableau-export-2.0.csv')

Removing unnecessary countries

In [ ]:
countries = ['GHA', 'RWA', 'UGA', 'CIV', 'KEN']
df = df[df['Country'].isin(countries)]

Removing unnecessary rounds

In [ ]:
df = df[~df['Round'].isin(['Onboarding', '6', '6.0']) & df['Round'].notna()]
df['Round'] = pd.to_numeric(df['Round'], errors='coerce')
df = df[df['Round'] % 1 == 0]
df['Round'] = df['Round'].astype(int).astype(str)
df = df[df['Round'].isin(['0', '1', '2', '3', '100', '102'])]
df['Round'] = df['Round'].astype(float).astype(int).astype(str)
df = df.sort_values(by='Round', ascending=True)

print(df['Round'].unique())

Removing unnecessary columns

In [ ]:
columns = [
    "Groupnr",
    "Round",
    "Country",
    "childmortality",
    "childmortalitytime",
    *[f"foodsecurity{i}" for i in range(1, 10)],
    *[f"foodsecurity{i}freq" for i in range(1, 10)],
    "fuelcooking",
    "sourcelighting",
    "watersource",
    "timewatersource_1",
    "timewatersourceunit",
    "Toiletfacility",
    "materialroof",
    "materialfloor",
    "materialwallsext",
    "assetsmatrix2_7",
    "assetsmatrix2_14",
    "assetsmatrix2_16",
    "assetsmatrix1_23",
    "assetsmatrix3_14",
    "assetsmatrix3_16",
    "assetsmatrix2_12",
    "assetsmatrix3_22",
    *[f"HHMschool_{n}" for n in range(1, 6)],
    *[f"HHMschoolnow_{n}" for n in range(1, 6)],
    *[f"HHMschoolcompl_{n}" for n in range(1, 6)],
    "school",
    "schoolcompleted",
    "savinghowmuch_1",
    "savinghowmuch_2",
    "savinghowmuch_3",
    "savingstotal_1",
    "debt",
    "debtamount_1",
    "debtnote",
    *[f"anxiety{i}" for i in range(1, 8)],
    "psychwellbeing_1",
    "psychwellbeing_3",
    "psychwellbeing_5",
    "psychwellbeing2_5",
    "jealousy",
    "jealousywhat",
    *[f"livestocknumbers_{i}" for i in [1,13,3,4,5,6,11,8,9,7,2,10]],
    "assetsmatrix1_4",
    "assetsmatrix1_5",
    "assetsmatrix1_22",
    "assetsmatrix2_7",
    "assetsmatrix2_14",
    "assetsmatrix2_15",
    "assetsmatrix2_16",
    "assetsmatrix2_8",
    "assetsmatrix3_17",
    "assetsmatrix2_17",
    "assetsmatrix2_18",
    "assetsmatrix2_19",
    "assetsmatrix2_11",
    "assetsmatrix2_12",
    "assetsmatrix3_14",
    "assetsmatrix1_23",
    "assetsmatrix3_15",
    "assetsmatrix3_16",
    "assetsmatrix3_22",
    "assetsmatrix3_23",
    "occupationmain",
    "ownsland_scto",
    "meetings1",
    "moneywithdraw",
    "moneyproblems"
]

columns_available_in_data = [col for col in columns if col in df.columns]

df = df[columns_available_in_data]


Setting datatypes

In [ ]:
numerical = [
    "savinghowmuch_1", "savinghowmuch_2", "savinghowmuch_3",
    "savingstotal_1", "debtamount_1", "timewatersource_1"
]


ordered_categorical = [
    *[f"foodsecurity{i}freq" for i in range(1, 10)],
    *[f"anxiety{i}" for i in range(1, 8)],
    "psychwellbeing_1", "psychwellbeing_3", "psychwellbeing_5", "psychwellbeing2_5"
]


categorical = [
    "fuelcooking", "sourcelighting", "watersource", "Toiletfacility",
    "materialroof", "materialfloor", "materialwallsext",
    *[f"HHMschoolcompl_{n}" for n in range(1, 6)],
    "schoolcompleted", "livestocknumbers_1",
    *[f"livestocknumbers_{i}" for i in [1, 13, 3, 4, 5, 6, 11, 8, 9, 7, 2, 10]],
    "occupationmain"
]


binary = [
    "childmortality",
    *[f"foodsecurity{i}" for i in range(1, 10)],
    *[f"HHMschool_{n}" for n in range(1, 6)],
    *[f"HHMschoolnow_{n}" for n in range(1, 6)],
    "school", "debt", "jealousy",
    "assetsmatrix1_4", "assetsmatrix1_5", "assetsmatrix1_22",
    "assetsmatrix2_15", "assetsmatrix2_8", "assetsmatrix3_17",
    "assetsmatrix2_17", "assetsmatrix2_18", "assetsmatrix2_19",
    "assetsmatrix2_11", "assetsmatrix3_15", "assetsmatrix3_23",
    "meetings1", "moneywithdraw", "moneyproblems"
]


multiple_choice = [
    "debtnote", "jealousywhat"
]


information = [
    "Country", "Groupnr", "Round"
]

binary_neg = [
    "debt", "foodsecurity1","foodsecurity2", "foodsecurity3", "foodsecurity4", "foodsecurity5",
    "foodsecurity6", "foodsecurity7", "foodsecurity8", "foodsecurity9", "childmortality",
    "jealousy", "assetsmatrix1_4", "assetsmatrix1_5", "assetsmatrix1_22", "assetsmatrix2_15",
    "assetsmatrix2_8", "assetsmatrix3_17", "assetsmatrix2_17",
    "assetsmatrix2_18", "assetsmatrix2_19", "assetsmatrix2_11",
    "assetsmatrix3_15", "assetsmatrix3_23"
]

binary_pos = [
    "HHMschoolnow_1", "HHMschoolnow_2", "HHMschoolnow_3",
    "HHMschoolnow_4", "HHMschoolnow_5",
    "school", "meetings1", "moneywithdraw", "moneyproblems"
]

Standardization functions of the categorical + binary + multiple choice variables  -- Work from K. Pereira

In [ ]:
def livestock_normal(val):
    if pd.isna(val) or val == 0: return 0
    val = int(val)
    if val in [1, 2, 3, 4, 5]: return str(val)
    elif 6 <= val <= 10: return '6-10'
    elif 11 <= val <= 20: return '11-20'
    elif val > 20: return '20+'
livestock_indices_normal = [1, 13, 3, 4, 8, 9, 7, 2, 10]
for i in livestock_indices_normal:
    col = f'livestocknumbers_{i}'
    if col in df.columns: df[col] = df[col].apply(livestock_normal)

def livestock_high(val):
    if pd.isna(val) or val == 0: return 0
    val = int(val)
    if val in [1, 2, 3, 4, 5]: return str(val)
    elif 6 <= val <= 10: return '6-10'
    elif 11 <= val <= 20: return '11-20'
    elif 21 <= val <= 50: return '21-50'
    elif val >= 51: return '51+'
    else: return 0

livestock_indices_high = [5,6,11]
for i in livestock_indices_high:
    col = f'livestocknumbers_{i}'
    if col in df.columns: df[col] = df[col].apply(livestock_high)

def mpi_water(row):
    if row['timewatersource_1'] > 30: return 1.0
    water = row['watersource']
    if water in [1, 2, 5, 7, 12]: return 0.0
    elif water in [4, 10]: return 0.3
    elif water in [3, 6]: return 0.6
    elif water in [8, 9, 11]: return 1.0
    else: return None
if 'timewatersource_1' in df.columns and 'watersource' in df.columns:
    df['MPI_water'] = df.apply(mpi_water, axis=1)

def MPI_fuel(row):
    fuel = row['fuelcooking']
    if fuel in [3, 4, 5, 6]: return 0.0
    elif fuel in [7]: return 0.3
    elif fuel in [2, 10]: return 0.6
    elif fuel in [1, 8, 9]: return 1.0
    else: return None
if 'fuelcooking' in df.columns:
    df['MPI_fuel'] = df.apply(MPI_fuel, axis=1)

def MPI_electricity(row):
    source = row['sourcelighting']
    if source in [1, 2, 4, 9, 15]: return 0.0
    elif source in [3]: return 0.3
    elif source in [5, 7, 8, 10, 13, 14]: return 0.6
    elif source in [6, 11, 12]: return 1.0
    else: return None
if 'sourcelighting' in df.columns:
    df['MPI_electricity'] = df.apply(MPI_electricity, axis=1)

def MPI_sanitation(row):
    toiletfacility = row['Toiletfacility']
    if toiletfacility in [1]: return 2
    elif toiletfacility in [6]: return 0.2
    elif toiletfacility in [5]: return 0.6
    elif toiletfacility in [3]: return 1.0
    else: return None
if 'Toiletfacility' in df.columns:
    df['MPI_sanitation'] = df.apply(MPI_sanitation, axis=1)

def MPI_floor(row):
    material = row['materialfloor']
    if material in [5, 6, 4, 9]: return 2
    elif material in [3, 8]: return 1
    elif material in [1, 2]: return 0
    else: return None

def MPI_roof(row):
    material = row['materialroof']
    if material in [4, 2, 3, 8]: return 2
    elif material in [5, 7]: return 1
    elif material in [1, 9]: return 0
    else: return None

def MPI_wall(row):
    material = row['materialwallsext']
    if material in [4, 6, 2, 3, 13]: return 2
    elif material in [1, 5, 8, 9, 11]: return 1
    elif material in [7, 14]: return 0
    else: return None

if 'materialwallsext' in df.columns: df['material_walls'] = df.apply(MPI_wall, axis=1)
if 'materialfloor' in df.columns: df['material_floor'] = df.apply(MPI_floor, axis=1)
if 'materialroof' in df.columns: df['material_roof'] = df.apply(MPI_roof, axis=1)

def average_material_score(row):
    values = [row.get('material_walls'), row.get('material_floor'), row.get('material_roof')]
    valid_values = [v for v in values if v is not None]
    return sum(valid_values) / len(valid_values) if valid_values else None
df['MPI_house'] = df.apply(average_material_score, axis=1)

def update_debtamount(row):
    if row['debt'] == 2.0: return 0
    elif row['debt'] == 1.0: return row['debtamount_1']
    else: return None
if 'debt' in df.columns and 'debtamount_1' in df.columns:
    df['debtamount_1'] = df.apply(update_debtamount, axis=1)

def calculate_school(row, n):
    age_col = f'HHMage_1_{n}'
    school_col = f'HHMschool_{n}'
    schoolcompl_col = f'HHMschoolcompl_{n}'
    if pd.isnull(row.get(age_col)) or row.get(age_col) < 10 or pd.isnull(row.get(school_col)): return None
    elif row.get(school_col) == 2: return 1
    elif pd.isnull(row.get(schoolcompl_col)): return 1
    elif row.get(schoolcompl_col) in [1, -88]: return 1
    else: return 0
for n in range(1, 21):
    if f'HHMage_1_{n}' in df.columns and f'HHMschool_{n}' in df.columns and f'HHMschoolcompl_{n}' in df.columns:
      df[f'MPI_6yearsofschool_perHHM_{n}'] = df.apply(lambda row: calculate_school(row, n), axis=1)

def MPI_6yearsofschool_woman(row):
    if pd.isnull(row.get('school')): return None
    elif row.get('school') == 2: return 1
    elif pd.isnull(row.get('schoolcompleted')): return 1
    elif row.get('schoolcompleted') in [1, -88]: return 1
    else: return 0
if 'school' in df.columns and 'schoolcompleted' in df.columns:
    df['MPI_6yearsofschool_woman'] = df.apply(MPI_6yearsofschool_woman, axis=1)

def MPI_6yearsofschool_allHHM(row):
    education_columns = [col for col in [f'MPI_6yearsofschool_perHHM_{i}' for i in range(1, 21)] + ['MPI_6yearsofschool_woman'] if col in row.index]
    if not education_columns or all(pd.isnull(row[col]) for col in education_columns): return None
    elif any(row[col] == 0 for col in education_columns): return 0
    else: return 1
df['MPI_6yearsofschool_allHHM'] = df.apply(MPI_6yearsofschool_allHHM, axis=1)

if 'occupationmain' in df.columns:
    occupation_scores = {0: 0, 6: 0, 4: 0, 31: 0, 16: 0, 18: 1, 54: 1, 20: 1, 8: 1, 36: 1, 56: 1, 39: 1, 48: 1, 7: 1, 5: 1, 47: 1, 17: 1, 50: 1, 40: 1, 15: 2, 37: 2, 55: 2, 57: 2, 58: 2, 52: 2, 49: 2}
    df['occupation_score'] = df['occupationmain'].map(occupation_scores)

# =============================================================================
# === THIS IS THE CORRECTED FUNCTION ===
# =============================================================================
def map_school_level_to_score(value):
    # Convert value to numeric. If it can't be converted, it becomes NaN.
    value = pd.to_numeric(value, errors='coerce')

    # The rest of the logic now works safely because `value` is either a number or NaN.
    if pd.isnull(value) or value in [-88, 1, 2]:
        return 0  # bad
    elif value in [3, 4]:
        return 1  # medium
    elif value >= 5:
        return 2  # good
    else:
        return 0  # fallback for other numbers or cases

school_columns = [f"HHMschoolcompl_{n}" for n in range(1, 6)] + ["schoolcompleted"]
for col in school_columns:
    if col in df.columns:
        df[f"{col}_score"] = df[col].apply(map_school_level_to_score)

if 'jealousywhat' in df.columns:
    def map_jealousy_score(code):
        if code in [3, 4, 5, 6, 7]: return 0
        elif code in [1, 2, 8]: return 1
        elif code == 0: return 2
        else: return None
    df['jealousywhat_score'] = df['jealousywhat'].apply(map_jealousy_score)

Ensuring numerical values

In [ ]:
import pandas as pd

def convert_to_numeric_except_id_columns(df):
    """
    Converts all columns in the dataframe to numeric values, coercing errors to NaN,
    except for 'Groupnr' and 'Country' which are left unchanged.

    Parameters:
    df (pd.DataFrame): Input dataframe.

    Returns:
    pd.DataFrame: DataFrame with specified columns converted to numeric where applicable.
    """
    df_copy = df.copy()
    cols_to_convert = [col for col in df_copy.columns if col not in ["Groupnr", "Country"]]

    for col in cols_to_convert:
        df_copy[col] = pd.to_numeric(df_copy[col], errors="coerce")

    return df_copy


Taking mean group values and adding dummy rows for missing rounds

In [ ]:
import pandas as pd

def add_missing_rounds(df):
    """
    Ensures each group in the dataframe has all required round numbers
    [0, 1, 2, 3, 100, 102]. For any missing round, a dummy row is inserted.

    Parameters:
    df (pd.DataFrame): Input dataframe containing at least 'Groupnr' and 'Round' columns.

    Returns:
    pd.DataFrame: DataFrame with dummy rows added for missing rounds per group.
    """
    required_rounds = [0, 1, 2, 3, 100, 102]
    all_rows = []

    for group_id, group_df in df.groupby("Groupnr"):
        existing_rounds = group_df["Round"].unique().tolist()
        missing_rounds = [r for r in required_rounds if r not in existing_rounds]

        all_rows.append(group_df)

        for missing_round in missing_rounds:
            dummy_row = {col: None for col in df.columns}
            dummy_row["Groupnr"] = group_id
            dummy_row["Round"] = missing_round
            all_rows.append(pd.DataFrame([dummy_row]))

    return pd.concat(all_rows, ignore_index=True).sort_values(by=["Groupnr", "Round"]).reset_index(drop=True)


# Modelling

Seperating into country dataframes

In [ ]:
df_gha = df[df['Country'] == 'GHA']
df_rwa = df[df['Country'] == 'RWA']
df_uga = df[df['Country'] == 'UGA']
df_civ = df[df['Country'] == 'CIV']
df_ken = df[df['Country'] == 'KEN']

Creating benchmark rows

In [ ]:
import pandas as np
import pandas as pd

def add_benchmark_rows(df):
    """
    Adds benchmark rows to the dataframe and fills them with mean values per round,
    excluding 'Groupnr', 'Country', and 'Round'.
    """
    benchmark_rows = [
        "benchmark_baseline",
        "benchmark_phone_survey_1",
        "benchmark_phone_survey_2",
        "benchmark_phone_survey_3",
        "benchmark_endline",
        "benchmark_post-program_survey_2"
    ]
    round_mapping = {
        "benchmark_baseline": 0,
        "benchmark_phone_survey_1": 1,
        "benchmark_phone_survey_2": 2,
        "benchmark_phone_survey_3": 3,
        "benchmark_endline": 100,
        "benchmark_post-program_survey_2": 102
    }
    df_copy = df.copy()
    for row_name in benchmark_rows:
        df_copy.loc[row_name] = np.nan
        df_copy.loc[row_name, "Round"] = round_mapping[row_name]
        df_copy.loc[row_name, "Groupnr"] = "benchmark"
        df_copy.loc[row_name, "Country"] = df["Country"].mode().iloc[0] if "Country" in df.columns else "unknown"

    for row_name in benchmark_rows:
        round_number = round_mapping[row_name]
        df_round = df[df["Round"] == round_number]
        mean_values = df_round.drop(columns=["Groupnr", "Country", "Round"]).mean(numeric_only=True)
        for col, value in mean_values.items():
            df_copy.loc[row_name, col] = value

    return df_copy.reset_index(drop=True)


Raw data plot function with IQR zone and dotted lines for missing values + plots average growth

In [ ]:
import matplotlib.pyplot as plt

def plot_groups_vs_overall(df, groups, variables):
    """
    Plots the trends of selected groups against overall average and IQR bands for given variables.
    """
    round_order = [0, 1, 2, 3, 100, 102]
    labels = {
        0: 'Baseline',
        1: 'Phone survey 1',
        2: 'Phone survey 2',
        3: 'Phone survey 3',
        100: 'Endline',
        102: 'Post-program survey'
    }
    label_list = [labels[r] for r in round_order]

    df = df.copy()
    df['Round'] = df['Round'].astype(int)

    plt.style.use('ggplot')
    cmap = plt.colormaps['tab10']

    for var in variables:
        fig, ax = plt.subplots(figsize=(10, 6), dpi=120)

        grouped = df.groupby('Round')[var]
        q1 = grouped.quantile(0.25).reindex(round_order)
        q3 = grouped.quantile(0.75).reindex(round_order)
        iqr_lower = (q1 - 1.5 * (q3 - q1)).values
        iqr_upper = (q3 + 1.5 * (q3 - q1)).values
        x_labels = [labels[r] for r in round_order]

        ax.fill_between(x_labels, iqr_lower, iqr_upper, color='grey', alpha=0.3, label='IQR band (±1.5×)')
        ax.plot(x_labels, iqr_lower, color='black', linewidth=1, label='IQR lower')
        ax.plot(x_labels, iqr_upper, color='black', linewidth=1, label='IQR upper')

        overall = (
            df.groupby('Round')[var]
              .mean()
              .reindex(round_order)
              .reset_index()
        )
        overall['lbl'] = overall['Round'].map(labels)
        ax.plot(overall['lbl'], overall[var],
                linestyle='--', marker='o', color='tab:red',
                linewidth=2, markersize=8, label='Overall average')

        for i, grp in enumerate(groups):
            sub = df[df['Groupnr'] == grp][['Round', var]].dropna()
            if sub.empty:
                continue

            pts = sorted(zip(sub['Round'], sub[var]), key=lambda x: round_order.index(x[0]))
            rounds, vals = zip(*pts)
            lbls = [labels[r] for r in rounds]

            for j in range(1, len(rounds)):
                r0, r1 = rounds[j-1], rounds[j]
                x = [labels[r0], labels[r1]]
                y = [vals[j-1], vals[j]]
                idx0 = round_order.index(r0)
                idx1 = round_order.index(r1)
                style = '-' if (idx1 - idx0 == 1) else 'dotted'
                ax.plot(x, y, linestyle=style, color=cmap(i), linewidth=1.5)

            ax.scatter(lbls, vals, marker='s', s=50, color=cmap(i), label=f'Group {grp}')

        ax.set_title(f'Original data of {var} for {grp} vs overall', fontsize=16, pad=12)
        ax.set_xlabel('Survey Round', fontsize=14, labelpad=8)
        ax.set_ylabel(var, fontsize=14, labelpad=8)
        ax.set_xticks(label_list)
        ax.set_xticklabels(label_list, rotation=45, ha='right')
        ax.grid(alpha=0.3)
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.legend(loc='upper left', bbox_to_anchor=(1.02, 1))
        plt.tight_layout()
        plt.show()


Plots the original data growth and IQR zone + plots average growth

In [ ]:
import matplotlib.pyplot as plt

def growth_plot(df, groups, variables):
    """
    For each var in `variables`, plots:
      - Grey-shaded IQR band (±1.5×IQR) and its boundary lines
      - Dotted red line = overall mean growth per round
      - Each group's growth trajectory (solid between true neighbors, dotted for gaps)
    """
    round_order = [0, 1, 2, 3, 100, 102]
    round_labels = {
        0:  'Baseline',
        1:  'Phone survey 1',
        2:  'Phone survey 2',
        3:  'Phone survey 3',
        100:'Endline',
        102:'Post-program survey'
    }
    x_pos = list(range(len(round_order)))
    x_labels = [round_labels[r] for r in round_order]

    df = df.copy()
    df['Round'] = df['Round'].astype(int)

    plt.style.use('ggplot')
    cmap = plt.colormaps['tab10']

    for var in variables:
        growth_col = var + '_growth'
        if growth_col not in df.columns:
            print(f"No growth column for {var}, skipping.")
            continue

        stats = (
            df.groupby('Round')[growth_col]
              .agg(Q1=lambda x: x.quantile(0.25),
                   Q3=lambda x: x.quantile(0.75))
              .reindex(round_order)
        )
        stats['IQR']   = stats['Q3'] - stats['Q1']
        stats['lower'] = stats['Q1'] - 1.5 * stats['IQR']
        stats['upper'] = stats['Q3'] + 1.5 * stats['IQR']

        overall = (
            df.groupby('Round')[growth_col]
              .mean()
              .reindex(round_order)
        )

        fig, ax = plt.subplots(figsize=(10,6), dpi=120)

        ax.fill_between(
            x_pos,
            stats['lower'],
            stats['upper'],
            color='gray',
            alpha=0.3,
            label='IQR band (±1.5×)'
        )
        ax.plot(x_pos, stats['lower'], color='black', lw=1.5, label='IQR lower')
        ax.plot(x_pos, stats['upper'], color='black', lw=1.5, label='IQR upper')

        ax.plot(
            x_pos, overall,
            linestyle=':', marker='o', color='red',
            lw=2, markersize=8,
            label='Avg growth'
        )

        for i, grp in enumerate(groups):
            sub = (
                df[df['Groupnr'] == grp]
                  [['Round', growth_col]]
                  .dropna()
                  .query("Round in @round_order")
            )
            if sub.empty:
                continue

            sub = sub.set_index('Round').reindex(round_order).dropna().reset_index()
            rs = sub['Round'].tolist()
            ys = sub[growth_col].tolist()
            xs = [round_order.index(r) for r in rs]

            for j in range(1, len(xs)):
                dx = xs[j] - xs[j-1]
                style = '-' if dx == 1 else 'dotted'
                ax.plot(
                    [xs[j-1], xs[j]],
                    [ys[j-1], ys[j]],
                    linestyle=style,
                    color=cmap(i),
                    lw=1.5
                )

            ax.scatter(
                xs, ys,
                marker='s', s=60,
                color=cmap(i),
                label=f'Group {grp}'
            )

        ax.set_xticks(x_pos)
        ax.set_xticklabels(x_labels, rotation=45, ha='right')
        ax.set_xlabel('Survey Round')
        ax.set_ylabel(f'{var} growth (absolute Δ)')
        ax.set_title(f'Growth outliers: {var}', fontsize=16, pad=12)
        ax.grid(alpha=0.3)
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.legend(loc='upper left', bbox_to_anchor=(1.02, 1))
        plt.tight_layout()
        plt.show()


Basic outlier detection code

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def outlier_growth(df, groups, variables):
    """
    Detects outliers in growth values per variable and round using IQR thresholds.
    If outliers are found, plots growth and raw value comparisons for the flagged groups.
    """
    for var in variables:
        growth_col = var + '_growth'
        if growth_col not in df.columns:
            print(f"No column {growth_col} found.")
            continue

        stats = df.groupby('Round')[growth_col].agg(
            Q1=lambda x: x.quantile(0.25),
            Q3=lambda x: x.quantile(0.75)
        ).reset_index()
        stats['IQR']   = stats['Q3'] - stats['Q1']
        stats['lower'] = stats['Q1'] - 1.5 * stats['IQR']
        stats['upper'] = stats['Q3'] + 1.5 * stats['IQR']

        df_stats = df.merge(stats, on='Round', how='left')

        print(f"\n- Outliers for {growth_col} per round -")
        flagged_groups = set()

        for grp in groups:
            sub = df_stats[df_stats['Groupnr'] == grp]
            for _, row in sub.iterrows():
                val = row[growth_col]
                if pd.isna(val):
                    continue
                if val < row['lower']:
                    print(f"Group {grp}, Round {row['Round']}: NEGATIVE outlier ({val:.2f} < {row['lower']:.2f})")
                    flagged_groups.add(grp)
                elif val > row['upper']:
                    print(f"Group {grp}, Round {row['Round']}: POSITIVE outlier ({val:.2f} > {row['upper']:.2f})")
                    flagged_groups.add(grp)

        if flagged_groups:
            sorted_flagged = sorted(flagged_groups)
            print(f"\nPlot for variable {var} (only outlier‐groups: {sorted_flagged})")
            growth_plot(df, sorted_flagged, [var])
            plot_groups_vs_overall(df, sorted_flagged, [var])
        else:
            print(f"No outliers were found for {var}, skipping plot.")



Adding moving averages

In [ ]:

def add_sma_columns(
    df: pd.DataFrame,
    variables: list[str] | None = None,
    window: int = 3,
    min_periods: int = 1
) -> pd.DataFrame:
    df_sma = df.copy()

    if variables is None:
        numeric_cols = df_sma.select_dtypes(include='number').columns.tolist()
        variables = [col for col in numeric_cols if col not in ('Round',)]

    for var in variables:
        sma_col = f"{var}_SMA_{window}"
        df_sma[sma_col] = (
            df_sma
            .groupby('Groupnr')[var]
            .transform(lambda s: s.rolling(window=window, min_periods=min_periods).mean())
        )
    return df_sma

In [ ]:
def add_cma_columns(
    df: pd.DataFrame,
    variables: list[str] | None = None,
    window: int = 3,
    min_periods: int = 1) -> pd.DataFrame:

    df_cma = df.copy()

    if variables is None:
        numeric_cols = df_cma.select_dtypes(include='number').columns.tolist()
        variables = [col for col in numeric_cols if col != 'Round']

    for var in variables:
        cma_col = f"{var}_CMA_{window}"
        df_cma[cma_col] = (
            df_cma
            .groupby('Groupnr')[var]
            .transform(lambda s: s.rolling(window=window, min_periods=min_periods, center=True).mean())
        )
    return df_cma

In [ ]:
def add_wma_columns(df: pd.DataFrame,
                    variables: list | None = None,
                    window: int = 3,
                    weights: np.ndarray | None = None,
                    min_periods: int = 1) -> pd.DataFrame:
    df_wma = df.copy()

    if variables is None:
        numeric = df_wma.select_dtypes(include='number').columns.tolist()
        variables = [c for c in numeric if c not in ('Round',)]

    if weights is None:
        weights = np.arange(1, window + 1)
    weights = np.array(weights)
    if weights.shape[0] != window:
        raise ValueError("Length of weights not equal to the window size.")

    def wma_calc(x):
      x = x[~np.isnan(x)]
      k = len(x)
      if k == 0:
          return np.nan
      w = weights[-k:]
      return np.dot(x, w) / w.sum()

    for var in variables:
        colname = f"{var}_WMA_{window}"
        df_wma[colname] = (
            df_wma
              .groupby('Groupnr')[var]
              .transform(lambda s:
                  s.rolling(window=window, min_periods=min_periods, center=False)
                   .apply(wma_calc, raw=True)
              )
        )
    return df_wma

In [ ]:

def add_ewma_columns(
    df: pd.DataFrame,
    variables: list[str] | None = None,
    span: int = 3,
    adjust: bool = True,
    min_periods: int = 1
) -> pd.DataFrame:
    df_ewma = df.copy()

    if variables is None:
        numeric_cols = df_ewma.select_dtypes(include='number').columns.tolist()
        variables = [col for col in numeric_cols if col != 'Round']

    for var in variables:
        ewma_col = f"{var}_EWMA_{span}"
        df_ewma[ewma_col] = (
            df_ewma
            .groupby('Groupnr')[var]
            .transform(lambda s: s.ewm(span=span, adjust=adjust, min_periods=min_periods).mean())
        )
    return df_ewma

Adding tend detection models

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from scipy.stats import ttest_ind, kendalltau

round2week = {0: 0, 1: 25, 2: 50, 3: 75, 100: 100, 102: 150}
df_ewma_growth['Week'] = df_ewma_growth['Round'].map(round2week)

variables = ['savinghowmuch_1', 'savinghowmuch_2', 'debtamount_1']
groups = df_ewma_growth['Groupnr'].unique()

results = []

for grp in groups:
    df_grp = df_ewma_growth[df_ewma_growth['Groupnr'] == grp]
    for var in variables:
        df_tmp = df_grp[['Week', var]].dropna()
        if len(df_tmp) < 3:
            continue

        X = sm.add_constant(df_tmp['Week'])
        y = df_tmp[var].values

        ols = sm.OLS(y, X).fit()
        ols_slope, ols_p, ols_r2 = ols.params['Week'], ols.pvalues['Week'], ols.rsquared

        gls = sm.GLS(y, X).fit()
        gls_slope, gls_p = gls.params['Week'], gls.pvalues['Week']

        early = df_tmp[df_tmp['Week'] <= 50][var]
        late  = df_tmp[df_tmp['Week'] > 50][var]
        if len(early) >= 2 and len(late) >= 2:
            tas_stat, tas_p = ttest_ind(early, late, nan_policy='omit')
        else:
            tas_stat, tas_p = np.nan, np.nan

        tau, tau_p = kendalltau(df_tmp['Week'], y)
        trend_dir = 'up' if tau > 0 else ('down' if tau < 0 else 'flat')

        results.append({
            'Groupnr': grp,
            'Variable': var,
            'OLS_slope': ols_slope,
            'OLS_p': ols_p,
            'OLS_R2': ols_r2,
            'GLS_slope': gls_slope,
            'GLS_p': gls_p,
            'TAS_stat': tas_stat,
            'TAS_p': tas_p,
            'Kendall_tau': tau,
            'Kendall_p': tau_p,
            'Trend_dir': trend_dir
        })

trend_df = pd.DataFrame(results)
trend_df


# Evaluation

Testing the different Moving Averages by plotting

In [ ]:
import matplotlib.pyplot as plt

def compare_smoothing(
    df_dict,
    groups,
    variables,
    window_size=3
):
    round_order = [0, 1, 2, 3, 100, 102]
    round_labels = ['Baseline', 'Phone 1', 'Phone 2', 'Phone 3', 'Endline', 'Post-program']
    round_map = dict(zip(round_order, round_labels))

    for variable in variables:
        n_groups = len(groups)
        fig, axes = plt.subplots(nrows=1, ncols=n_groups, figsize=(6 * n_groups, 5), sharey=True)

        if n_groups == 1:
            axes = [axes]

        for i, group in enumerate(groups):
            ax = axes[i]

            base_df = next(iter(df_dict.values()))
            raw = base_df[(base_df['Groupnr'] == group) & (base_df['Round'].isin(round_order))].copy()
            raw['RoundLabel'] = raw['Round'].map(round_map)
            ax.plot(raw['RoundLabel'], raw[variable], linestyle='--', marker='o', label='Raw')

            for method, df in df_dict.items():
                colname = f"{variable}_{method.upper()}_{window_size}"
                if colname in df.columns:
                    sub = df[(df['Groupnr'] == group) & (df['Round'].isin(round_order))].copy()
                    sub['RoundLabel'] = sub['Round'].map(round_map)
                    ax.plot(sub['RoundLabel'], sub[colname], label=method.upper())

            ax.set_title(f'Group {group}')
            ax.set_xlabel('Survey Round')
            if i == 0:
                ax.set_ylabel(variable)
            ax.grid(True)
            ax.legend()

        plt.suptitle(f"Smoothing comparison for '{variable}'", fontsize=16)
        plt.tight_layout()
        plt.show()

Evaluating trend detection models with plots

Violin p value plots

In [ ]:
plt.figure(figsize=(10, 6))
sns.violinplot(
    data=trend_df,
    x="OLS_p",
    y="Variable",
    hue="Variable",
    palette="Dark2",
    split=False,
    inner="stick",
    legend=False
)

plt.axvline(
    0.05,
    color="red",
    linestyle="--",
    linewidth=2,
    label="p = 0.05"
)

plt.title("Violin of OLS p-values by Variable")
plt.legend(loc="upper right")
plt.tight_layout()
plt.show()


TAS p values

In [ ]:

plt.figure(figsize=(8, 4))
sns.boxplot(
    data=trend_df,
    y="Variable",
    x="TAS_p",
    hue='Variable',
    palette="Set2",
    showfliers=True,
    flierprops={"marker":"o","alpha":0.4,"markersize":4},
    legend=False
)
plt.axvline(0.05, color="red", linestyle="--")
plt.title("TAS p-value by Variable")
plt.tight_layout()
plt.show()

Counts for Kendalls tau

In [ ]:
plt.figure(figsize=(8, 6))
order = ["up", "flat", "down"]
ax = sns.countplot(
    data=trend_df,
    x="Trend_dir",
    order=order,
    hue="Variable",
    palette="Set2"
)
plt.title("Count of Trend Directions by Variable")
plt.ylabel("Count")


plt.legend(
    loc="upper center",
    bbox_to_anchor=(0.5, -0.15),
    ncol=3,
    frameon=False
)

plt.tight_layout()
plt.show()

OLS vs GLS slopes

In [ ]:
g = sns.FacetGrid(trend_df, col="Variable", height=4, aspect=1)
g.set_titles("{col_name}")
g.map_dataframe(
    sns.scatterplot,
    x="OLS_slope",
    y="GLS_slope",
    alpha=0.7
)

for ax in g.axes.flat:
    lim = max(
        abs(ax.get_xlim()[0]), ax.get_xlim()[1],
        abs(ax.get_ylim()[0]), ax.get_ylim()[1]
    )
    ax.plot([-lim, lim], [-lim, lim],
            ls="--", lw=1.5, c="blue")
    ax.set_aspect("equal", "box")

g.set_axis_labels("OLS slope", "GLS slope")
g.fig.suptitle("OLS vs GLS slopes, by Variable", y=1.02)
plt.tight_layout()
plt.show()


#Results

Plotting function of EWMA-based IQR zone compared to original growth and average growth

In [ ]:
def growth_plot_strict_iqr(
    df: pd.DataFrame,
    groups: list[str],
    var: str,
    methods: list[str] = ['EWMA','WMA'],
    window: int = 3
):
    """
    Plots a strict intersection-based IQR band across multiple smoothing methods
    and visualizes group growth trends against this threshold.
    """
    import numpy as np
    import matplotlib.pyplot as plt

    round_order = [0,1,2,3,100,102]
    labels = {
        0:'Baseline',1:'Phone survey 1',2:'Phone survey 2',
        3:'Phone survey 3',100:'Endline',102:'Post-program survey'
    }
    x = np.arange(len(round_order))
    xlabs = [labels[r] for r in round_order]

    stats = {}
    for m in methods:
        gc = f"{var}_{m.upper()}_{window}_growth"
        s = df.groupby('Round')[gc].agg(
            Q1=lambda x: x.quantile(0.25),
            Q3=lambda x: x.quantile(0.75)
        ).reindex(round_order)
        s['lower'] = s['Q1'] - 1.5*(s['Q3']-s['Q1'])
        s['upper'] = s['Q3'] + 1.5*(s['Q3']-s['Q1'])
        stats[m] = s

    lowers = np.vstack([stats[m]['lower'] for m in methods])
    uppers = np.vstack([stats[m]['upper'] for m in methods])
    strict_lower = np.max(lowers, axis=0)
    strict_upper = np.min(uppers, axis=0)

    raw_gc = var + '_growth'
    overall = df.groupby('Round')[raw_gc].mean().reindex(round_order)

    plt.style.use('ggplot')
    fig, ax = plt.subplots(figsize=(10,5), dpi=120)

    ax.fill_between(x, strict_lower, strict_upper,
                    color='lightgray', alpha=0.6,
                    label='IQR band (strict ∩ EWMA)')

    ax.plot(x, strict_lower,   'k-', lw=1, label='IQR lower')
    ax.plot(x, strict_upper,   'k-', lw=1, label='IQR upper')

    ax.plot(x, overall.values, 'o--', color='red', lw=2, ms=6,
            label='Avg growth')

    cmap = plt.get_cmap('tab10')
    for i, gid in enumerate(groups):
        ax.plot([], [],
                color=cmap(i),
                marker='s',
                linestyle='-',
                linewidth=1.5,
                label=f"Group {gid}")

    def plot_group(gid, color):
        sub = (df[df['Groupnr']==gid]
               .set_index('Round')[raw_gc]
               .reindex(round_order))
        y = sub.values
        for j in range(1,len(x)):
            ls = '-' if (not np.isnan(y[j-1]) and not np.isnan(y[j])) else ':'
            ax.plot(x[j-1:j+1], y[j-1:j+1],
                    color=color, lw=1.5, linestyle=ls)
        ax.scatter(x, y, color=color, s=60, marker='s')

    for i, gid in enumerate(groups):
        plot_group(gid, cmap(i))

    ax.set_xticks(x)
    ax.set_xticklabels(xlabs, rotation=45, ha='right')
    ax.set_ylabel(f"{var}_growth (absolute Δ)")
    ax.set_title(f"Growth outliers: {var}")
    ax.legend(loc='upper left', bbox_to_anchor=(1.02,1))
    plt.tight_layout()
    plt.show()


Outlier detection using the new EWMA based IQR zone

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def outlier_growth_ewma(
    df: pd.DataFrame,
    groups: list[str],
    variables: list[str],
    window: int = 3
):
    """
    For each variable in `variables`, compute the IQR band from its *_EWMA_{window}_growth series,
    then flag *raw* var_growth whenever it steps outside that band.
    Finally, for any variable with at least one flag, plot:
      1) growth_plot_strict_iqr (the IQR‐band view)
      2) plot_groups_vs_overall (the raw trend)
    """
    round_order = [0, 1, 2, 3, 100, 102]

    flagged_per_var = {var: set() for var in variables}

    for var in variables:
        gc = f"{var}_EWMA_{window}_growth"
        if gc not in df.columns:
            raise ValueError(f"Missing column {gc}")
        grp = df.groupby("Round")[gc]
        s = grp.agg(Q1=lambda x: x.quantile(0.25),
                    Q3=lambda x: x.quantile(0.75)).reindex(round_order)
        iqr = s["Q3"] - s["Q1"]
        band = pd.DataFrame({
            "lower": s["Q1"] - 1.5 * iqr,
            "upper": s["Q3"] + 1.5 * iqr
        }, index=round_order)

        raw_gc = var + "_growth"
        if raw_gc not in df.columns:
            print(f"Warning: no raw growth column {raw_gc}")
            continue

        tmp = df[["Groupnr", "Round", raw_gc]].merge(
            band.reset_index().rename(columns={"index": "Round"}),
            on="Round", how="left"
        )

        print(f"\n- EWMA-IQR outliers for {var}_growth -")
        for grp in groups:
            sub = tmp[tmp["Groupnr"] == grp]
            for _, r in sub.iterrows():
                v = r[raw_gc]
                if pd.isna(v):
                    continue
                if v < r["lower"]:
                    print(f"! NEG outlier: {var}, Group {grp}, Round {r['Round']}: {v:.1f} < {r['lower']:.1f}")
                    flagged_per_var[var].add(grp)
                elif v > r["upper"]:
                    print(f"! POS outlier: {var}, Group {grp}, Round {r['Round']}: {v:.1f} > {r['upper']:.1f}")
                    flagged_per_var[var].add(grp)

    for var, fl in flagged_per_var.items():
        if not fl:
            print(f"\n– No outliers found for {var}.")
            continue

        fl = sorted(fl)
        print(f"\n! Plotting EWMA-IQR band & raw trends for {var} (groups: {fl}) !")
        growth_plot_strict_iqr(df, fl, var, methods=["EWMA"], window=window)
        plot_groups_vs_overall(df, fl, [var])
